In [1]:
import os


folder_path = '/Users/rumszz/Documents/GitHubRepos/Shceduling_AI/application_source/data/raw_2'

files = os.listdir(folder_path)

print(files)

# # Change working directory
os.chdir(folder_path)
# # Check current working directory
print(os.getcwd())

['enhanced_sales_meeting_history.csv', 'inventory_mapping.csv', 'customer_data.csv', 'customer_to_sales_rep.csv']
/Users/rumszz/Documents/GitHubRepos/Shceduling_AI/application_source/data/raw_2


In [ ]:
#pip install pandas


In [ ]:
#pip install scikit-learn

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
today = pd.to_datetime("2025-05-08")

In [13]:


# Load customer data and meeting history
customer_df = pd.read_csv("customer_data.csv")
meeting_df = pd.read_csv("enhanced_sales_meeting_history.csv")

In [14]:
# 1. Compute last meeting date per customer

meeting_df["Meeting Timestamp"] = pd.to_datetime(
    meeting_df["Meeting Timestamp"], format="%Y-%m-%d %I:%M %p"
)
latest_meetings = meeting_df.groupby("Customer ID")["Meeting Timestamp"].max().reset_index()
latest_meetings.columns = ["Customer ID", "Last Meeting Date"]
latest_meetings["Last Meeting Date"] = pd.to_datetime(latest_meetings["Last Meeting Date"])

In [15]:
# 2. Merge with customer data
customer_df = customer_df.drop(['Last Meeting Date'], axis=1)
customer_df["Last Purchase Date"] = pd.to_datetime(customer_df["Last Purchase Date"], format="%Y-%m-%d")
df = customer_df.merge(latest_meetings, on="Customer ID", how="left")

In [16]:
# 3. Compute days since last meeting and purchase

df["Days Since Last Meeting"] = (today - df["Last Meeting Date"]).dt.days
df["Days Since Last Purchase"] = (today - df["Last Purchase Date"]).dt.days


In [17]:
# 4. Merge in recent meeting outcomes
latest_outcomes = meeting_df.sort_values("Meeting Timestamp").groupby("Customer ID").tail(1)[["Customer ID", "Meeting Outcome"]]
df = df.merge(latest_outcomes, on="Customer ID", how="left")
df["Needs Follow-up"] = df["Meeting Outcome"].apply(lambda x: 1 if "follow-up" in str(x).lower() or "approval" in str(x).lower() else 0)

In [18]:
# 5. Normalize scoring columns
scaler = MinMaxScaler()
score_components = df[["Days Since Last Meeting", "Days Since Last Purchase", "Purchase Freq/Q", "Last 12M Revenue ($)", "Needs Follow-up"]].copy()
score_components["Purchase Freq/Q"] = 1 / (score_components["Purchase Freq/Q"] + 1e-5)  # Invert freq so low = higher score
score_components["Last 12M Revenue ($)"] = 1 / (score_components["Last 12M Revenue ($)"] + 1e-5)  # Invert to prioritize low-engaged high-value customers
normalized = pd.DataFrame(scaler.fit_transform(score_components), columns=score_components.columns)

In [19]:
# 6. Apply weights and compute priority score
weights = {
    "Days Since Last Meeting": 0.3,
    "Days Since Last Purchase": 0.3,
    "Purchase Freq/Q": 0.1,
    "Last 12M Revenue ($)": 0.2,
    "Needs Follow-up": 0.1
}
df["Priority Score"] = sum(normalized[col] * weight for col, weight in weights.items())

In [20]:
# 7. For each sales rep, get top 2 customers
top_priority = df.sort_values(["Sales Rep ID", "Priority Score"], ascending=[True, False]) \
                 .groupby("Sales Rep ID").head(2)[["Sales Rep ID", "Customer ID", "Customer Name", "Priority Score"]]

In [22]:
top_priority.to_csv("priority_df.csv", index=False)